In [8]:
import torch
import librosa
import numpy as np
import sys
print("imports complete")

imports complete


In [6]:
MODEL_PATH = "best_combined_model.pth"
CLASSES = ["angry", "disgust", "fear", "happy", "neutral", "sad"]
SAMPLE_RATE = 16000
N_FFT = 2048
HOP_LENGTH = 512
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def load_model(path):
    model = torch.load(path, map_location=DEVICE)
    model.eval()
    return model

def process_audio(file_path):
    y, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    stft = np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH))
    spec_db = librosa.amplitude_to_db(stft, ref=np.max)
    spec_db = (spec_db - spec_db.mean()) / (spec_db.std() + 1e-6)
    spec_tensor = torch.tensor(spec_db).unsqueeze(0).unsqueeze(0).float().to(DEVICE)
    return spec_tensor

def classify(model, spec_tensor):
    with torch.no_grad():
        outputs = model(spec_tensor)
        predicted = torch.argmax(outputs, dim=1).item()
    return CLASSES[predicted]

if __name__ == "__main__":
    audio_path = "crema-d-miror/AudioMP3/1001_DFA_ANG_XX.mp3"
    model = load_model(MODEL_PATH)
    spec_tensor = process_audio(audio_path)
    prediction = classify(model, spec_tensor)

    print(f"predicted class: {prediction}")

AttributeError: 'dict' object has no attribute 'eval'